In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from matplotlib import pyplot as plt
import math
from sklearn.model_selection import GridSearchCV

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
with open('/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat '/gdrive/My Drive/foo.txt'

In [0]:
def transformDataset(dataset):
    dataset['Dates'] = pd.to_datetime(dataset['Dates'])
    
    dataset['Date'] = dataset['Dates'].dt.date
    
    dataset['n_days'] = (dataset['Date'] - dataset['Date'].min()).apply(lambda x: x.days)
    
    dataset['Year'] = dataset['Dates'].dt.year
    dataset['DayOfWeek'] = dataset['Dates'].dt.dayofweek # OVERWRITE
    dataset['WeekOfYear'] = dataset['Dates'].dt.weekofyear
    dataset['Month'] = dataset['Dates'].dt.month
    
    dataset['Hour'] = dataset['Dates'].dt.hour
    
    # dataset['Block'] = dataset['Address'].str.contains('block', case=False)
    # dataset['Block'] = dataset['Block'].map(lambda x: 1 if  x == True else 0)

    
    dataset = dataset.drop('Dates', 1)
    dataset = dataset.drop('Date', 1)
    dataset = dataset.drop('Address', 1)
    
    # dataset = pd.get_dummies(data=dataset, columns=[ 'PdDistrict'], drop_first = True)
    district = dataset["PdDistrict"].unique()
    data_dict_district = {}
    count = 1
    for data in district:
        data_dict_district[data] = count
        count+=1
    dataset['PdDistrict'] = dataset['PdDistrict'].replace(data_dict_district)
    
    return dataset


In [0]:
def llfun(act, pred):
    """ Logloss function for 1/0 probability
    """
    return (-(~(act == pred)).astype(int) * math.log(1e-15)).sum() / len(act)

In [0]:
sf_crime_train = pd.read_csv("/gdrive/My Drive/ALDA Project/train.csv")
sf_crime_train.drop(['Descript', 'Resolution'], axis=1, inplace=True)

sf_crime_test = pd.read_csv("/gdrive/My Drive/ALDA Project/test.csv")

sf_crime_train = transformDataset(sf_crime_train)
data_dict = {}
count = 1
for data in sf_crime_train["Category"].unique():
    data_dict[data] = count
    count+=1
sf_crime_train["Category"] = sf_crime_train["Category"].replace(data_dict)
sf_crime_test = transformDataset(sf_crime_test)

x = sf_crime_train.drop(columns=['Category'])
y = sf_crime_train['Category'].values

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
with open('/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat '/gdrive/My Drive/foo.txt'

Hello Google Drive!

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1, stratify=y)

In [0]:
logloss = []
i = 1
counter = 1
while i<len(x_train):
    print("Itearation" +str(counter))
    print("Neighbours"+str(i))
    knn = KNeighborsClassifier(n_neighbors=i, n_jobs=10)
    knn.fit(np.array(x_train), np.array(y_train))
    # Predict on test set
    outcome = knn.predict(x_test)
    # Logloss
    logloss.append(llfun(y_test, outcome))
    i*=2
    counter+=1
    print(logloss[-1])
    if len(logloss)>=2 and logloss[-1]>logloss[-2]:
        break

Itearation1
Neighbours1
27.461483929820616
Itearation2
Neighbours2


KeyboardInterrupt: ignored

In [0]:
# knn = KNeighborsClassifier(n_neighbors=40, n_jobs=10)
# knn.fit(np.array(x_train), np.array(y_train))
# outcome = knn.predict(x_test)

In [0]:
# knn.score(x_test, y_test)

0.20767040601332498

In [0]:
knn2 = KNeighborsClassifier(n_jobs=10)
param_grid = {'n_neighbors': np.arange(30, 60)}
knn_gscv = GridSearchCV(knn2, param_grid, cv=5)
knn_gscv.fit(x, y)

In [0]:
knn = KNeighborsClassifier(n_neighbors=59, n_jobs=10)
knn.fit(np.array(x_train), np.array(y_train))
outcome = knn.predict(x_test)
outcome

array([ 3,  3,  3, ...,  2,  2, 14])

In [0]:
for a,b in zip(outcome,y_test):
  if a==b:
    counter+=1


In [0]:
##KNN Accuracy using Normal KNN
print(counter/len(outcome))

0.20788110016513867


In [0]:
## Applying KNN along with Kernel Function:  

knn = KNeighborsClassifier(n_neighbors=59, n_jobs=10,weights='distance')
knn.fit(np.array(x_train), np.array(y_train))
outcome = knn.predict(x_test)




In [0]:
## Two Percent Imrpovement upon using  a Kernel function  
counter = 0
for a,b in zip(outcome,y_test):
  if a==b:
    counter+=1
print(counter/len(outcome))



##Using Different Kernel functions:


0.2217755253117704
